In [ ]:
# default_exp benchmarking

In [ ]:
#export
import matplotlib.pyplot as plt
import numpy as np


def plot_lines(protvals, log = True):
    """plot peptide profiles for a protein"""
    colors = plt.cm.tab20c(np.linspace(0,1,protvals.shape[0]))
    #cmap = plt.get_cmap("tab20c")
    idx = 0
    for row in protvals:
        if not log:
            row = 2**row
        plt.plot(row, c= colors[idx])
        idx+=1
    median_row = np.nanmedian(protvals, axis=0)
    print(median_row)
    plt.plot(median_row, c = 'black',linewidth =3 )
    plt.show()

def plot_points(protvals, log = True):
    colors = plt.cm.tab20c(np.linspace(0,1,protvals.shape[0]))
    #cmap = plt.get_cmap("tab20c")
    idx = 0
    for row in protvals:
        if not log:
            row = 2**row
        x_coord = list(range(len(row)))
        plt.scatter(x_coord,row, c= colors[idx])
        idx+=1

    plt.show()

In [ ]:
#export
import pandas as pd
def get_tps_fps(result_df, prot2org_file, thresh = 0.05, fc_thresh = 0.3):
    annotated = annotate_dataframe(result_df, prot2org_file)
    condpairs = result_df["condpair"].drop_duplicates()


    for condpair in condpairs:
        annotated_condpair = annotated[annotated["condpair"]==condpair]
        num_tps = sum(annotated_condpair["TP"])
        num_fps = sum(annotated_condpair["FP"])
        annotated_fcfilt = annotated_condpair[annotated["log2fc"] >fc_thresh]
        num_regulated_prots = sum(annotated_fcfilt["fdr"]<thresh)
        num_true_positives = sum(annotated_fcfilt["TP"] &(annotated_fcfilt["fdr"]<0.05))
        num_false_positives = sum(annotated_fcfilt["FP"] &(annotated_fcfilt["fdr"]<0.05))
        fpr = num_false_positives/num_regulated_prots

        print(f'condpair {condpair}')
        print(f"total TPs {num_tps}")
        print(f"total FPs {num_fps}")
        print(f'regulated {num_regulated_prots}')
        print(f'false positives {num_false_positives}')
        print(f'true positives {num_true_positives}')
        print(f'regulated control {num_false_positives+num_true_positives}')
        print(f'FPR {fpr}')

        assert fpr < 0.06


def annotate_dataframe(result_df, prot2org_file):
    prot2org = pd.read_csv(prot2org_file, sep = "\t")
    prot2org["FP"] = (prot2org["organism"] == "Homo sapiens")
    prot2org["TP"] = (prot2org["organism"] == "Saccharomyces cerevisiae")
    prot2org = prot2org[(prot2org["FP"] | prot2org["TP"])]
    print(f"df size before {len(result_df.index)}")
    annotated = pd.merge(result_df, prot2org, how='inner', on = "protein")
    print(f"df size after {len(annotated.index)}")
    return annotated

In [ ]:
#export
import pandas as pd
import matplotlib.pyplot as plt
def compare_to_reference(result_df, reference_file, condpair):#put in condpair as tuple
    result_df = result_df[result_df["condpair"]==condpair]

    ref_df = pd.read_csv(reference_file, sep = "\t")
    merged = pd.merge(result_df, ref_df, how='inner', on = "protein",suffixes = ["", "_ref"])
    ax_p = merged.plot.scatter(x='pval_ref',y='pval')
    plt.show()
    ax_fc = merged.plot.scatter(x='log2FC_ref',y='fc')
    plt.show()
    ax_fdr = merged.plot.scatter(x='fdr_ref',y='fdr')
    plt.show()

In [ ]:
#export
import pandas as pd
import matplotlib.pyplot as plt
def compare_normalization(ref_normalization_file, norm1_df, norm2_df):
    ref_normed = pd.read_csv(ref_normalization_file, sep ="\t").set_index('peptide')

    merged = pd.merge(norm1_df, norm2_df, how='inner',  left_index = True, right_index = True)
    columns = merged.columns
    merged = pd.merge(ref_normed, merged, how='inner', left_index = True, right_index = True, suffixes = ["_ref", ""])

    for i in range(len(columns)):
        sample1 = columns[i]
        sample2 = sample1+"_ref"
        ax_p = merged.plot.scatter(x=sample1,y=sample2)
        plt.show()

In [ ]:
#export
def print_nonref_hits(protein_ref, protein_df, peptide_ref, peptide_df, outdir):
    prots_nonref_df =  protein_df[~(protein_df["protein"].isin(protein_ref["protein"].to_list()))]#the tilde inverts the boolean vector
    peps_nonref_df = peptide_df[~(peptide_df["peptide"].isin(peptide_ref["peptide"].to_list()))]
    prots_nonref_df.to_csv(f"{outdir}/nonref_proteins.tsv", sep = "\t", index = False)
    peps_nonref_df.to_csv(f"{outdir}/nonref_peptides.tsv", sep = "\t", index = False)
    #display(peps_nonref_df)

# Utility functions to compare against other methods

# System-wide benchmark

# Prepare Benchmarking Sets

# Spike-in Benchmarks

## Ratio Comparisons

## Classification tests

In [ ]:
#export
import pandas as pd
import functools


class ResultsTable():
    def __init__(self, input_file, input_name):
        self._input_file = input_file
        self._samples_c1 = None
        self._samples_c2 = None
        self.input_name = input_name
        self.protein_column = "protein"
        self.organism_column = "organism"
        self.log2fc_column = 'log2fc'
        self.mean_intensity_column = 'mean_intensity'
        self.formated_dataframe = None

    
    def get_proteins(self):
        return self.formated_dataframe[self.protein_column]

    def _add_mean_intensity_column(self):
        self._add_median_intensity_columns_for_each_condition()
        self.formated_dataframe[self.mean_intensity_column] = self.formated_dataframe[['median_intensity_c1', 'median_intensity_c2']].mean(axis = 1)

    
    def _add_median_intensity_columns_for_each_condition(self):
        self.formated_dataframe['median_intensity_c1'] = self.formated_dataframe[self._samples_c1].median(axis =1)
        self.formated_dataframe['median_intensity_c2'] = self.formated_dataframe[self._samples_c2].median(axis =1)

    def _add_log2fc_column(self):
        self.formated_dataframe[self.log2fc_column] = [np.log2(x[0]) - np.log2(x[1]) for x in zip(self.formated_dataframe['median_intensity_c1'], self.formated_dataframe['median_intensity_c2'])]




class ResultsTableDirectLFQ(ResultsTable):
    def __init__(self, input_file, input_name,  samples_c1, samples_c2):
        super().__init__(input_file, input_name)
        self._samples_c1 = samples_c1
        self._samples_c2 = samples_c2
        self._define_formatted_dataframe()
        
    def _define_formatted_dataframe(self):
        self._load_directlfq_output()
        self._add_median_intensity_columns_for_each_condition()
        self._add_log2fc_column()
        self._add_mean_intensity_column()
        
    def _load_directlfq_output(self):
        self.formated_dataframe =  pd.read_csv(self._input_file, sep = "\t")



class ResultsTableMaxQuant(ResultsTable):
    def __init__(self, input_file, input_name,  samples_c1, samples_c2):
        super().__init__(input_file, input_name)
        self._samples_c1 = samples_c1
        self._samples_c2 = samples_c2
        self._define_formatted_dataframe()

    
    def _define_formatted_dataframe(self):
        self._load_maxquant_output()
        self._add_median_intensity_columns_for_each_condition()
        self._add_log2fc_column()
        self._add_mean_intensity_column()

    def _load_maxquant_output(self):
        columns_to_use = self._samples_c1 + self._samples_c2 + ["id"]
        self.formated_dataframe = pd.read_csv(self._input_file, sep = "\t", usecols=columns_to_use)
        self.formated_dataframe = self.formated_dataframe.rename({'id':self.protein_column},axis=1)




In [ ]:
#export

class OrganismAnnotator():
    def __init__(self, mapping_file, protein_column = 'id', organism_colum = 'Species'):
        self._mapping_file = mapping_file
        self._protein_column = protein_column
        self._organism_column = organism_colum
        self._protein_organism_mapping_df = self.__load_reduce_mapping_dataframe()

    def annotate_table_with_organism(self, results_table):
        organism_column = results_table.organism_column
        protein_column = results_table.protein_column

        results_df = self.__add_organism_column(results_table.formated_dataframe, organism_column, protein_column)
        results_df = self.__filter_non_matching_proteins(results_df, organism_column)
        
        results_table.formated_dataframe = results_df

    def save_protein_organism_map(self, outfile):
        self._protein_organism_mapping_df.to_csv(outfile, sep = "\t", index = None)
    
    
    def __load_reduce_mapping_dataframe(self):
        mapping_df = pd.read_csv(self._mapping_file, sep = "\t", usecols=[self._protein_column, self._organism_column], encoding='latin1').drop_duplicates()
        mapping_df = self.__filter_double_mapping_organism(mapping_df)
        return mapping_df
    
    def __filter_double_mapping_organism(self, protein2organism_df):
        protein2organism_df = protein2organism_df[[";" not in x for x in protein2organism_df[self._organism_column].astype('str')]] #a semicolon seperates different organism entries
        return protein2organism_df
    

    def __add_organism_column(self, results_df,organism_column, protein_column):
        protein2organism_dict = self.__get_protein2organism_dict()
        results_df[organism_column] = [protein2organism_dict.get(x) for x in results_df[protein_column].astype('str')]
        return results_df

    def __get_protein2organism_dict(self):
        protein2organism = dict(zip(self._protein_organism_mapping_df[self._protein_column].astype('str'), self._protein_organism_mapping_df[self._organism_column]))
        return protein2organism
    
    @staticmethod
    def __filter_non_matching_proteins(results_df, organism_column):
        results_df = results_df[[x is not None for x in results_df[organism_column]]]
        return results_df
        

In [ ]:
#export
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt


class PlotConfig():
    def __init__(self, x_axisboundaries = None, y_axisboundaries = None, colormaps = None):
        self.x_axisboundaries = x_axisboundaries
        self.y_axisboundaries = y_axisboundaries
        self._colormaps = colormaps

class MultiOrganismIntensityFCPlotter():
    def __init__(self, ax, resultstable_w_ratios, plotconfig = PlotConfig()):
        self.ax = ax
        self._resultstable_w_ratios = resultstable_w_ratios
        self._organism_column = resultstable_w_ratios.organism_column
        self._log2fc_column = resultstable_w_ratios.log2fc_column
        self._mean_intensity_column = resultstable_w_ratios.mean_intensity_column
        self._plotconfig = plotconfig
        
        self._all_organisms = self._get_all_organisms_used()
        self._title = self._get_title()
        self._scatter_per_organism()
        

    def _get_all_organisms_used(self):
        return sorted(list(set(self._resultstable_w_ratios.formated_dataframe[self._organism_column].astype('str'))))
    
    def _get_title(self):
        title = ""
        for organism in self._all_organisms:
            subtable_organism = self._get_organism_subtable(organism)
            title += self._extend_title(organism,subtable_organism)
        return title

    def _scatter_per_organism(self):
        complete_table = self._resultstable_w_ratios.formated_dataframe.copy()
        complete_table[self._mean_intensity_column] = np.log2(complete_table[self._mean_intensity_column])
        sns.scatterplot(data= complete_table, x = self._log2fc_column, y=self._mean_intensity_column, hue=self._organism_column, alpha=0.15, ax=self.ax, hue_order=self._all_organisms)
        self.ax.set_title(self._title)
        self.ax.set_xlim(self._plotconfig.x_axisboundaries)

    def _get_organism_subtable(self, organism):
        complete_table = self._resultstable_w_ratios.formated_dataframe
        return complete_table[complete_table[self._organism_column] == organism]
    
    def _extend_title(self, organism, subtable_organism):
        fcs = subtable_organism[self._log2fc_column].to_numpy()
        fcs = fcs[np.isfinite(fcs)]
        median_fc = np.nanmedian(fcs)
        std_fc = np.nanstd(fcs)
        num_ratios = sum(~np.isnan(fcs))
        return f"{organism} num:{num_ratios} median_FC:{median_fc:.2} STD:{std_fc:.2}\n"
 


def scatter_annotated_ratioresults(alphaquant_results_file):
    results_ratios = aqbench.ResultsTableRatios(alphaquant_results_file, input_name="AlphaQuant")
    mapping_file = "mapping_file.tsv"

    speciesannotator = aqbench.SpeciesAnnotator(mapping_file=mapping_file, protein_column="Protein.Group", species_colum="Organism")
    speciesannotator.annotate_table_with_species(results_ratios)
    ax = plt.axes()
    plotconfig  = PlotConfig(x_axisboundaries=[-4, 4])

    MultiOrganismIntensityFCPlotter(ax,results_ratios,plotconfig=plotconfig)
    ax.get_figure().savefig(f"{alphaquant_results_file}.ratio_plots.pdf", bbox_inches = 'tight')
    return ax


### Learning Tests

In [15]:
import seaborn as sns
import numpy as np


def check_taking_mean_of_pandas_columns():
    df = sns.load_dataset('iris')
    display(df)
    df.iloc[3:6, 1] = np.nan
    display(df)
    df['mean_sepal_width_petal_length'] = df[["sepal_width", "petal_length"]].median(axis = 1, skipna = True)
    assert df.loc[3, 'mean_sepal_width_petal_length'] == df.loc[3, 'petal_length']


check_taking_mean_of_pandas_columns()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,NaN,1.5,0.2,setosa
4,5.0,NaN,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica
